In [3]:
using MixedLayerModel
using FileIO

In [6]:
# set-up basic 400 ppm simulation
par = basic_params();
par.rtype = varRad();
par.etype = Sally();
par.a = 1.1;
path = "./output/Sally1.1/";

# run to steady-state
u0, sol = run_mlm(par);

# get output
code = sol.retcode;
println(code);
uf = sol.u;
du = zeros(4);
mlm(du, uf, par, 0.0);
zi,hM,qM,SST = uf;
println(uf);
zb = calc_LCL(zi,hM,qM);

# save to file
output = Dict("code" => code, "p"=>par, "u0" => u0, "uf" => uf, "du/u" => du./uf, 
"we" => we(uf,par,par.etype), "zb" => zb, "zc" => zi-zb,
"RHsurf" => RH(0.0, hM, qM), "LHF" => calc_LHF(uf,par), "SHF" => calc_SHF(uf,par),
"ΔR" => calc_cloudtop_RAD(uf,par,par.rtype), "OHU" => calc_OHU(uf,par,par.stype))
save(path*"co2_400.jld2", output)

Success
[1150.920076973418, 312618.51064756373, 0.007981982898737713, 290.0]


In [10]:
##########
# increase CO2 but fix SST
##########

# set co2
newCO2 = 600.0

# load initial condition from file
path = "./output/Sally1.1/";
output = load(path*"co2_400.jld2");
u0 = output["uf"];
OHU = output["OHU"];

# increase CO2, let SST evolve and check cloud changes
par.CO2 = newCO2;
u0, sol = run_mlm_from_init(u0, par);

# get output
code = sol.retcode;
println(code);
uf = sol.u;
du = zeros(4);
mlm(du, uf, par, 0.0);
zi,hM,qM,SST = uf;
zb = calc_LCL(zi,hM,qM);

# save to file
output = Dict("code" => code, "p" => par, "u0" => u0, "uf" => uf, "du/u" => du./uf, 
"we" => we(uf,par,par.etype), "zb" => zb, "zc" => zi-zb,
"RHsurf" => RH(0.0, hM, qM), "LHF" => calc_LHF(uf,par), "SHF" => calc_SHF(uf,par),
"ΔR" => calc_cloudtop_RAD(uf,par,par.rtype), "OHU" => calc_OHU(uf,par,par.stype))
save(path*"co2_upstep_fixSST_"*string(Int(newCO2))*".jld2", output)

Success


In [ ]:
##########
# increase CO2 and let SST evolve
##########

# load initial condition from file
output = load(path*"co2_400.jld2");
u0 = output["uf"];
OHU = output["OHU"];

# set OHU, let SST evolve and check cloud changes
newCO2 = 600.0;
par.CO2 = newCO2;
par.Hw = 10;
par.OHU = OHU;
par.stype = varSST();
u0, sol = run_mlm_from_init(u0, par);

# get output
code = sol.retcode;
println(code);
uf = sol.u;
du = zeros(4);
mlm(du, uf, par, 0.0);
zi,hM,qM,SST = uf;
zb = calc_LCL(zi,hM,qM);

# save to file
output = Dict("code" => code, "p" => par, "u0" => u0, "uf" => uf, "du/u" => du./uf, 
"we" => we(uf,par,par.etype), "zb" => zb, "zc" => zi-zb,
"RHsurf" => RH(0.0, hM, qM), "LHF" => calc_LHF(uf,par), "SHF" => calc_SHF(uf,par),
"ΔR" => calc_cloudtop_RAD(uf,par,par.rtype), "OHU" => calc_OHU(uf,par,par.stype))
save(path*"co2_upstep_"*string(Int(newCO2))*".jld2", output)